<a href="https://colab.research.google.com/github/Bast1-py/Retrieval-Augmented-Generation-Vector-Database-to-Store-Document-Embeddings/blob/main/Retrieval_Augmented_Generation_Vector_Database_to_Store_Document_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import Image, HTML, display, display_html
source_tautan = "https://skills.network/logos/SN_web_lightmode.png"
HTML(f'<div style="text-align: center;"><img src="{source_tautan}" style="max-width: 520px; height: auto;"></div>')

# **Introduction:**

In [ ]:
from IPython.display import Image, HTML, display, display_html
source_tautan = "https://media.licdn.com/dms/image/v2/D4D22AQFBPLeHjncRVg/feedshare-shrink_2048_1536/B4DZdaT_Q4GUA4-/0/1749566882317?e=1752710400&v=beta&t=_EeOByam5k5KWc8Y_9wYEmaL1z25mrNN_FAE3Nw7UJQ"
HTML(f'<div style="text-align: center;"><img src="{source_tautan}" style="max-width: 710px; height: auto;"></div>')

In [ ]:
from IPython.display import Image, HTML, display, display_html
source_tautan = "https://media.licdn.com/dms/image/v2/D4D22AQHsKoZTkx9Vkw/feedshare-shrink_800/B4DZdcc.y1HwAk-/0/1749602792393?e=1752710400&v=beta&t=VN_U-Y3uHX_AN6nm5phAhY_J0O1_byziiS6wyg1bi4E"
HTML(f'<div style="text-align: center;"><img src="{source_tautan}" style="max-width: 680px; height: auto;"></div>')



---


# Create and Configure a Vector Database to Store Document Embeddings.


---



  Bayangkan jika anda bekerja di customer support center yang menerima banyak pertanyaan dan tiket setiap hari. Tugas anda adalah membuat sistem yang dapat dengan cepat menyediakan informasi yang paling relevan kepada agen dukungan untuk menyelesaikan masalah customer.

  Metode Tradisional untuk mencari melalui FAQs atau dokumen dukungan dapat berjalan lambat dan kuran efisien yang bisa mengakibatkan respon banyak tertunda dan customer tidak puas.

  Untuk mengatasi tantangan ini, anda akan menggunakan model **Embedding** untuk mengubah **support documents** dan **past inquiry response** sebelumnya menjadi vektor numerik yang menangkap konten **semantic content.** Vektor ini akan disimpan dalam basis data vektor, yang memungkinkan pencarian **enabling fast** dan **accurate similarity.**

  Misalnya, ketika **Agent** dukungan menerima pertanyaan baru tentang masalah produk, sistem dapat langsung mengambil pertanyaan serupa sebelumnya dan penyelesaiannya, yang mampu membantu agent untuk memberikan respon yang lebih cepat dan lebih akurat.

In [ ]:
from IPython.display import Image, HTML, display, display_html
source_tautan = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/veZYoygp9GqZrIw5f6SD0g/vector%20db.png"
HTML(f'<div style="text-align: center;"><img src="{source_tautan}" style="max-width: 680px; height: auto;"></div>')

  untuk menyelesaikan kasus ini, disini akan mengerjakan cara menggunakan basis data vektor untuk menyimpan embedding yang dihasilkan dari data tekstual menggunakan **LangChain**.
  Fokus utamanya terletak pada dua basis data vektor populer. **Chroma DB** dan **Facebook AI Similarity Search (FAISS)**. Disini juga akan mempelajari cara melakukan pencarian silmilaritas di basis data ini berdasarkan kueri, yang memungkinkan pengambilan informasi secara relevan dan efisien. Disini juga dapat menggunakan **basis data vektor** secara efektif untuk **menyimpan dan mengkueri embedding**, yang **meningkatkan kemampuan anlisis dan pengambilan data**.

In [ ]:
from IPython.display import Image, HTML, display, display_html
source_tautan = "https://media.licdn.com/dms/image/v2/D5622AQEkg7qzXIYfRA/feedshare-shrink_1280/B56ZcQ..PAGsAk-/0/1748336635782?e=1752710400&v=beta&t=Ae3PvnYSlOx_xUJHt8H9GdTX_70IsV9mFebXeBLMQk4"
HTML(f'<div style="text-align: center;"><img src="{source_tautan}" style="max-width: 680px; height: auto;"></div>')

# Objectives:

  1. Prepare dan process document for **embeddings**.
  2. Membuat **embeddings** menggunakan **model watsonx.ai emdedding model.**
  3. Menyimpan **embeddings** ini ke **Chrome DB** dan **FAISS.**
  4. Melakukan Similarity Search untuk **retrieve relevant document** berdasarkan **new inquiries**

# Installing Required Libraries.


---



In [ ]:
# !pip install ibm-watsonx-ai
# !pip install langchain
# !pip install langchain-ibm
# !pip install langchain-cummunity
# !pip install chromedb
# !pip install faiss-cpu
# !pip install langchain_community



---


# Loading Data (textual)


---



In [ ]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/BYlUHaillwM8EUItaIytHQ/companypolicies.txt"

--2025-06-13 03:10:59--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/BYlUHaillwM8EUItaIytHQ/companypolicies.txt
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
200 OKequest sent, awaiting response... 
Length: 15660 (15K) [text/plain]
Saving to: ‘companypolicies.txt.1’

companypolicies.txt 100%[===================>]  15.29K  --.-KB/s    in 0s      

2025-06-13 03:10:59 (72.7 MB/s) - ‘companypolicies.txt.1’ saved [15660/15660]





---


# Import The Required Frameworks.


---



In [ ]:
import pandas as pd

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ibm import WatsonxEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames

In [ ]:
# loader = TextLoader('/content/companypolicies.txt')
loader = TextLoader('companypolicies.txt')
data = loader.load()
data

[Document(metadata={'source': 'companypolicies.txt'}, page_content="1.\tCode of Conduct\n\nOur Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.\nIntegrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.\nRespect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.\nAccountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to 

In [ ]:
def printData(data):
  for item in data:
    print(item)
printData(data)

page_content='1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.
Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.
Respect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.
Accountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential 



---


# Split Data


---





  untuk tahap ini sangat penting diterapkan sebagai split dokumen menggunakan LangChain's text splitter. Selanjutnya akan menggunakan **RecursiveCharacterTextSplitter** yang sangat cocok untuk teks generik. berikut parameter yang akan di set-up:

    * chunk_size = 100
    * chunck_overlap = 20
    * length_function = len

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,
                                               chunk_overlap=20,
                                               length_function=len)
chunks = text_splitter.split_documents(data)
chunks

[Document(metadata={'source': 'companypolicies.txt'}, page_content='1.\tCode of Conduct'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Our Code of Conduct outlines the fundamental principles and ethical standards that guide every'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='that guide every member of our organization. We are committed to maintaining a workplace that is'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='a workplace that is built on integrity, respect, and accountability.'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='acting honestly and transparently in all our interactions, whether with colleagues, clients, or the'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='clients, or the broader communi

In [ ]:
def printData(chunks):
  for item in chunks:
    print(item)
printData(chunks)

page_content='1.	Code of Conduct' metadata={'source': 'companypolicies.txt'}
page_content='Our Code of Conduct outlines the fundamental principles and ethical standards that guide every' metadata={'source': 'companypolicies.txt'}
page_content='that guide every member of our organization. We are committed to maintaining a workplace that is' metadata={'source': 'companypolicies.txt'}
page_content='a workplace that is built on integrity, respect, and accountability.' metadata={'source': 'companypolicies.txt'}
page_content='Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and' metadata={'source': 'companypolicies.txt'}
page_content='acting honestly and transparently in all our interactions, whether with colleagues, clients, or the' metadata={'source': 'companypolicies.txt'}
page_content='clients, or the broader community. We respect and protect sensitive information, and we avoid' metadata={'source': 'companypolicies.txt'}
page_content='and we avoid

In [ ]:
panjang_chunk = len(chunks)
print(f'Total Chunks Obtained From The Data: {panjang_chunk}')

Total Chunks Obtained From The Data: 215




---


# Embedding Model


---



  Pada tahap ini menunjukkan cara pembuatan **model embedding** menggunakan **watsonx.ai package**. Pada tugas ini, Embedding model **ibm/slate-125m-english-rtrvr** yang digunakan. Embedding model dibentuk menjadi obyek **watsonx_embedding**.



In [ ]:
embed_params = {EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS:3,
                EmbedTextParamsMetaNames.RETURN_OPTIONS: {'input_text': True},}

watsonx_embedding = WatsonxEmbeddings(model_id='ibm/slate-125m-english-rtrvr',
                                      url='https://us-south.ml.cloud.ibm.com',
                                      project_id='skills-network',
                                      params=embed_params,)



---


# Vector Store:


---
Pada bagian ini, akan menentukan cara penggunaan **dua basis data vektor** yang umum digunakan **Chroma DB** dan **FAISS DB** yang dilakukan untuk **similarity search** berdasarkan **input query** menggunakan basis data tersebut.




---


## 1. Chroma DB:


---
  bertujuan untuk membangun database dengan membutuhkan Chroma dari **penyimpanan vektor LangChain**.
  Namun, dibutuhkan membuat daftar ID yang akan digunakan untuk menetapkan pengenal unik pada setiap potongan, yang memungkinkan anda melacaknya nanti dalam **vector database**. pentih bahwa panjang daftar ini harus sesuai dengan panjang **chunks**

catatan bahwa **ID** harus dalam **string format.**

---
### 1. Build The Database
---

In [ ]:
ids = [str(i) for i in range(0, len(chunks))]

  langkah selanjutnya adalah menggunakan **embedding model** untuk **membuat embedding** untuk setiap **Chunks** dan kemudian menyimpannya dalam **Chroma Database.**

In [ ]:
vectordb = Chroma.from_documents(chunks, watsonx_embedding, ids=ids)
vectordb

  dari proses ini bisa **membangun vectore store** bernama **vectordb** seperti code diatas dan menggunakan **metode.collection.get()** untuk mencetak beberapa **chunks** yang diindeks berdasarkan **ID-nya**.


  perlu diketahui bahwa meskipun **chunks disimpan dalam database** dalam format embedding, saat mengambil dan **mencetak ID-nya**, database akan mengembalikan informasi **teks chunks** tersebut, bukan **vektor embedding**

In [ ]:
for i in range(5):
  print(vectordb._collection.get(ids=str(i)))

{'ids': ['0'], 'embeddings': None, 'metadatas': [{'source': 'companypolicies.txt'}], 'documents': ['1.\tCode of Conduct'], 'uris': None, 'data': None}
{'ids': ['1'], 'embeddings': None, 'metadatas': [{'source': 'companypolicies.txt'}], 'documents': ['Our Code of Conduct outlines the fundamental principles and ethical standards that guide every'], 'uris': None, 'data': None}
{'ids': ['2'], 'embeddings': None, 'metadatas': [{'source': 'companypolicies.txt'}], 'documents': ['that guide every member of our organization. We are committed to maintaining a workplace that is'], 'uris': None, 'data': None}
{'ids': ['3'], 'embeddings': None, 'metadatas': [{'source': 'companypolicies.txt'}], 'documents': ['a workplace that is built on integrity, respect, and accountability.'], 'uris': None, 'data': None}
{'ids': ['4'], 'embeddings': None, 'metadatas': [{'source': 'companypolicies.txt'}], 'documents': ['Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and']

  dari sini bahwa dapat menggunakan metode **_collection.count()** untuk melihat **panjang vector database**, yang seharusnya sama dengan **panjang chunks**.

In [ ]:
vectordb._collection.count()

215

---
### 2. Similarity Search
---
   Bagian Terpenting lainnya adalah, setelah **store vector database** adalah **Similarity Search** yang melibatkan **finding items** yang paling mirip dengan **query item** tertentu berdasarkan **representasi vektornya.**.

   Untuk proses ini, onjek data diubah menjadi vektor (dari proses yang telah dibuat sebelumnya), dan **search algorithm** mengidentifikasi dan mengambil objek dengan jarak vektor terdekat dengan kueri, yang memungkinkan identifikasi item serupa yang efisien dan akurat dalam kumpulan **large database.**

   **LangChain mendukung similarity search** dalam menyimpan vektor menggunakan metode **.similarity_search().**

   Berikut adalah contoh cara melakukan **similarity search** berdasarkan **kueari (Email policy).**

   secara default bahwa, ia akan mengembalikan empat vektor teratas yang paling dekat dengan **kueri.**

---
#### Example 1: **Email policy**
---

In [ ]:
query = "Email policy"
docs = vectordb.similarity_search(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='internet and email usage, including those related to copyright and data protection.'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='to this policy. Non-compliance may lead to appropriate disciplinary action, which could include'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='This policy serves as a framework for handling discipline and termination. The organization'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Policy Purpose: The Smoking Policy has been established to provide clear guidance and expectations')]

In [ ]:
print(f"Query: '{query}'")
print(f"Total Documents Found: {len(docs)} Similarity Documents")

Query: 'Email policy'
Total Documents Found: 4 Similarity Documents


In [ ]:
print("SIMILARITY SEARCH RESULTS:\n")
for i, doc in enumerate(docs, 1):
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content: {doc.page_content}")
    print('-'*110)

SIMILARITY SEARCH RESULTS:

Source: companypolicies.txt
Content: internet and email usage, including those related to copyright and data protection.
--------------------------------------------------------------------------------------------------------------
Source: companypolicies.txt
Content: to this policy. Non-compliance may lead to appropriate disciplinary action, which could include
--------------------------------------------------------------------------------------------------------------
Source: companypolicies.txt
Content: This policy serves as a framework for handling discipline and termination. The organization
--------------------------------------------------------------------------------------------------------------
Source: companypolicies.txt
Content: Policy Purpose: The Smoking Policy has been established to provide clear guidance and expectations
--------------------------------------------------------------------------------------------------------------


In [ ]:
vectordb.similarity_search(query, k=1)   # k=1 untuk hanya mengambil satu hasil teratas

[Document(metadata={'source': 'companypolicies.txt'}, page_content='internet and email usage, including those related to copyright and data protection.')]

---
## 2. Facebook AI Similarity Search (FIASS DB)
---

adalah **Vectore Database** lain yang didukung oleh **LangChain**. proses pembuatan dan penggunaan FAISS mirip dengan Chroma DB. namun, perbedaanya terletak pada hasil retrieval antara FAISS DB dan Chroma DB.

---
### 1. Build The Database:
---

In [ ]:
faissdb = FAISS.from_documents(chunks, watsonx_embedding, ids=ids)
faissdb

    Untuk tahap pertama adalah sama dengan Chroma DB, yaitu cetak tiga informasi pertama dalam database berdasarkan ID

In [ ]:
for i in range(3):
    print(vectordb._collection.get(ids=str(i)))

{'ids': ['0'], 'embeddings': None, 'metadatas': [{'source': 'companypolicies.txt'}], 'documents': ['1.\tCode of Conduct'], 'uris': None, 'data': None}
{'ids': ['1'], 'embeddings': None, 'metadatas': [{'source': 'companypolicies.txt'}], 'documents': ['Our Code of Conduct outlines the fundamental principles and ethical standards that guide every'], 'uris': None, 'data': None}
{'ids': ['2'], 'embeddings': None, 'metadatas': [{'source': 'companypolicies.txt'}], 'documents': ['that guide every member of our organization. We are committed to maintaining a workplace that is'], 'uris': None, 'data': None}


---
### 2. Similarity Search:
---

---
#### Example 2: **Smoking Police**

In [ ]:
query = "Smoking Policy"
docs = faissdb.similarity_search(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='Smoking Restrictions: Smoking inside company buildings, offices, meeting rooms, and other enclosed'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Designated Smoking Areas: Smoking is only permitted in designated smoking areas, as marked by'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='No Smoking in Company Vehicles: Smoking is not permitted in company vehicles, whether they are'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Policy Purpose: The Smoking Policy has been established to provide clear guidance and expectations')]

In [ ]:
print(f"Query: '{query}'")
print(f"Total Documents Found: {len(docs)} Similarity Documents:\n")

Query: 'Smoking Policy'
Total Documents Found: 4 Similarity Documents:



In [ ]:
print("SIMILARITY SEARCH RESULTS:\n")
for i, doc in enumerate(docs, 1):
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content: {doc.page_content}")
    print('-'*110)

SIMILARITY SEARCH RESULTS:

Source: companypolicies.txt
Content: Smoking Restrictions: Smoking inside company buildings, offices, meeting rooms, and other enclosed
--------------------------------------------------------------------------------------------------------------
Source: companypolicies.txt
Content: Designated Smoking Areas: Smoking is only permitted in designated smoking areas, as marked by
--------------------------------------------------------------------------------------------------------------
Source: companypolicies.txt
Content: No Smoking in Company Vehicles: Smoking is not permitted in company vehicles, whether they are
--------------------------------------------------------------------------------------------------------------
Source: companypolicies.txt
Content: Policy Purpose: The Smoking Policy has been established to provide clear guidance and expectations
--------------------------------------------------------------------------------------------------------

In [ ]:
vectordb.similarity_search(query, k=1)

[Document(metadata={'source': 'companypolicies.txt'}, page_content='Smoking Restrictions: Smoking inside company buildings, offices, meeting rooms, and other enclosed')]

---
## 3. Managing Vector Store: adding, updating, and deleting Entries.
---

Mungkin ada situasi dimana dokumen baru masuk ke **aplikasi RAG** anda yang ingin anda **tambahkan** ke database vektor saat ini, atau anda mungkin perlu **menghapus** beberapa dokumen yang ada dari database. selain itu, mungkin ada **pembaharuan** pada beberapa dokumen dalam basis data yang memerlukan **pembaruan.**

Maka pada tahap ini akan membuat cara mengelola Vector Store yang ingin di **add**, **delete**, dan **update** dan mempraktekkan dengan menggunakan **Chrome DB.**

---
### 1. Add
---
bayangkan jika anda memiliki informasi teks baru yang ingin ditambahkan ke database vektor. pertama, informasi ini harus diformat ke dalam objek dokumen.

In [ ]:
text = "Instructlab is the best open source tool for fine-tuning a LLM"
text

'Instructlab is the best open source tool for fine-tuning a LLM'

In [ ]:
new_chunk = Document(page_content=text,
                     metadata={"source": "ibm.com",
                               "page": 1})
new_chunk

Document(metadata={'source': 'ibm.com', 'page': 1}, page_content='Instructlab is the best open source tool for fine-tuning a LLM')

    Selanjutnya, new_chunk harus dimasukkan ke dalam daftar karena database vektor hanya menerima dokumen dalam daftar.

In [ ]:
new_chunk = [new_chunk]
new_chunk

[Document(metadata={'source': 'ibm.com', 'page': 1}, page_content='Instructlab is the best open source tool for fine-tuning a LLM')]

    Karena ada 215 chunks sebelumnya dengan ID dari 0 sampai 214, jika ditambahkan maka akan mencetak ID 215, dokumen tersebut seharusnya tidak menampilkan nilai apapun. mari divalidasi.

In [ ]:
print(vectordb._collection.get(ids=['215']))

{'ids': [], 'embeddings': None, 'metadatas': [], 'documents': [], 'uris': None, 'data': None}


Selanjutnya menggunakan metode **add_documents()** untuk menambahkan new_chunk. dalam metode ini, anda harus menetapkan ID ke dokumen tersebut. karena sudah ada ID dari 0 sampai 214, maka dapat d=menetapkan ID 215 ke dokumen ini. ID harus dalam format string dan ditempatkan dalam daftar.

In [ ]:
vectordb.add_documents(new_chunk, ids=['215'])

['215']

maka sekarang anda dapat menghitung panjang **vector database** untuk melihat apakah panjangnya bertambah satu.

In [ ]:
vectordb._collection.count()

216

In [ ]:
print(vectordb._collection.get(ids=['215']))

{'ids': ['215'], 'embeddings': None, 'metadatas': [{'page': 1, 'source': 'ibm.com'}], 'documents': ['Instructlab is the best open source tool for fine-tuning a LLM'], 'uris': None, 'data': None}


---
### 2. Update
---

In [ ]:
update_chunk = Document(page_content="Instructlab is a perfect open source tool for fine-tuning a LLM.",
                        metadata={"source": "ibm.com",
                                  "page": 1})

In [ ]:
vectordb.update_document('215', update_chunk)

In [ ]:
print(vectordb._collection.get(ids=['215']))

{'ids': ['215'], 'embeddings': None, 'metadatas': [{'page': 1, 'source': 'ibm.com'}], 'documents': ['Instructlab is a perfect open source tool for fine-tuning a LLM.'], 'uris': None, 'data': None}


---
### 3. Delete
---

In [ ]:
vectordb._collection.delete(ids=['215'])

In [ ]:
print(vectordb._collection.get(ids=['215']))

{'ids': [], 'embeddings': None, 'metadatas': [], 'documents': [], 'uris': None, 'data': None}
